# Production d'un csv utilisable de la base FINESS

En l'état, l'export CSV de la [base FINESS][finess] n'est pas vraiment satisfaisant et utilisable.

- Le fichier n'est pas réellement un CSV.
    - Il est bizarrement découpé en deux sections qui correspondent au XML.
    - Les colonnes n'ont pas de nom.
- Le fichier est encodé au format windows.

[finess]: https://www.data.gouv.fr/en/datasets/finess-extraction-du-fichier-des-etablissements/

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
dataset_api = "https://www.data.gouv.fr/api/1/datasets/finess-extraction-du-fichier-des-etablissements/"

In [3]:
resources = (requests
    .get(dataset_api)
    .json()
    ['resources']
)

resource_geoloc = [ r for r in resources if r['type'] == 'main' and 'géolocalisés' in r['title']][0]

In [4]:
headers = [
    'section',
    'nofinesset',
    'nofinessej',
    'rs',
    'rslongue',
    'complrs',
    'compldistrib',
    'numvoie',
    'typvoie',
    'voie',
    'compvoie',
    'lieuditbp',
    'commune',
    'departement',
    'libdepartement',
    'ligneacheminement',
    'telephone',
    'telecopie',
    'categetab',
    'libcategetab',
    'categagretab',
    'libcategagretab',
    'siret',
    'codeape',
    'codemft',
    'libmft',
    'codesph',
    'libsph',
    'dateouv',
    'dateautor',
    'maj',
    'numuai'
]

In [5]:
geoloc_names = [
    'nofinesset',
    'coordxet',
    'coordyet',
    'sourcecoordet',
    'datemaj'
]

In [6]:
raw_df = (pd
    .read_csv(resource_geoloc['url'],
              sep=";", encoding="Windows-1252", header=None, skiprows=1,
              dtype='str',
              names=headers)
    .drop(columns=['section'])
)

raw_df

,nofinesset,nofinessej,rs,rslongue,complrs,compldistrib,numvoie,typvoie,voie,compvoie,...,siret,codeape,codemft,libmft,codesph,libsph,dateouv,dateautor,maj,numuai
0,010000024,010780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,NaN,NaN,900,RTE,DE PARIS,NaN,...,26010004500012,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1979-02-13,1979-02-13,2020-02-04,NaN
1,010000032,010780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,NaN,NaN,700,AV,DE NARVIK,NaN,...,26010003700068,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2021-07-07,NaN
2,010000065,010780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,NaN,NaN,14,R,DE L'HOPITAL,NaN,...,26010028400017,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2018-01-12,NaN
3,010000081,010780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,NaN,NaN,160,R,MARC PANISSOD,NaN,...,26010010200011,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2020-02-04,NaN
4,010000099,010780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,NaN,NaN,13,AV,DU DOCTEUR BOYER,NaN,...,26010013600019,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1945-01-01,1945-01-01,2020-06-30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191487,980501738,521486.0,8583077.0,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191488,980501779,524168.6,8588089.7,"2,ATLASANTE,84,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191489,980501878,514247.9,8582244.9,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191490,980502199,522250.7,8592077.7,"3,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
structures = (raw_df
    .iloc[:int(raw_df.index.size/2)]
)

structures

,nofinesset,nofinessej,rs,rslongue,complrs,compldistrib,numvoie,typvoie,voie,compvoie,...,siret,codeape,codemft,libmft,codesph,libsph,dateouv,dateautor,maj,numuai
0,010000024,010780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,NaN,NaN,900,RTE,DE PARIS,NaN,...,26010004500012,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1979-02-13,1979-02-13,2020-02-04,NaN
1,010000032,010780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,NaN,NaN,700,AV,DE NARVIK,NaN,...,26010003700068,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2021-07-07,NaN
2,010000065,010780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,NaN,NaN,14,R,DE L'HOPITAL,NaN,...,26010028400017,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2018-01-12,NaN
3,010000081,010780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,NaN,NaN,160,R,MARC PANISSOD,NaN,...,26010010200011,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1901-01-01,1901-01-01,2020-02-04,NaN
4,010000099,010780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,NaN,NaN,13,AV,DU DOCTEUR BOYER,NaN,...,26010013600019,8610Z,03,ARS établissements Publics de santé dotation g...,1,Etablissement public de santé,1945-01-01,1945-01-01,2020-06-30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95741,980502389,590799730,SAMSAH TUTUBIYA - ALEFPA,NaN,NaN,NaN,390,R,DES FRANGIPANIERS,NaN,...,77562407502209,NaN,57,ARS /ARS PCD Dotation forfait ou prix de jou...,NaN,NaN,2022-06-08,2021-12-20,2022-07-19,NaN
95742,980502405,980500854,PLATEFORME SENSORIELLE - ADSM,NaN,NaN,NaN,4,R,DE LA SIM,NaN,...,NaN,NaN,34,ARS / DG dotation globale,NaN,NaN,2022-02-26,2021-04-12,2022-07-20,NaN
95743,980502439,980500557,SEPRODOM MAYOTTE,SEPRODOM MAYOTTE,NaN,NaN,NaN,CHE,MAVIKI,NaN,...,NaN,NaN,99,Indéterminé,0,Non concerné,2020-12-01,2020-10-13,2022-10-06,NaN
95744,980502454,980502447,LE CONFORT MEDICAL,NaN,NaN,NaN,100,RTE,DES BADAMIERS,C,...,NaN,NaN,99,Indéterminé,9,indéterminé,2022-09-16,2022-09-16,2022-10-06,NaN


In [8]:
geolocalisations = (raw_df
    .iloc[int(raw_df.index.size/2):]
    .drop(columns=raw_df.columns[5:])
    .rename(columns=lambda x: geoloc_names[list(raw_df.columns).index(x)])
)

geolocalisations

,nofinesset,coordxet,coordyet,sourcecoordet,datemaj
95746,020002978,714412.4,6946033.2,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
95747,020012779,736596.2,6900631.2,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
95748,030002208,705086.4,6591177.1,"3,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
95749,040000069,923147.4,6321804.0,"3,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
95750,040002313,950414.1,6331695.3,"3,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
...,...,...,...,...,...
191487,980501738,521486.0,8583077.0,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04
191488,980501779,524168.6,8588089.7,"2,ATLASANTE,84,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04
191489,980501878,514247.9,8582244.9,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04
191490,980502199,522250.7,8592077.7,"3,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04


In [9]:
clean_df = (structures
    .merge(geolocalisations, on="nofinesset", how="left")
)

clean_df

,nofinesset,nofinessej,rs,rslongue,complrs,compldistrib,numvoie,typvoie,voie,compvoie,...,codesph,libsph,dateouv,dateautor,maj,numuai,coordxet,coordyet,sourcecoordet,datemaj
0,010000024,010780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,NaN,NaN,900,RTE,DE PARIS,NaN,...,1,Etablissement public de santé,1979-02-13,1979-02-13,2020-02-04,NaN,870215.7,6571590.5,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
1,010000032,010780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,NaN,NaN,700,AV,DE NARVIK,NaN,...,1,Etablissement public de santé,1901-01-01,1901-01-01,2021-07-07,NaN,908351.7,6520414.1,"2,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
2,010000065,010780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,NaN,NaN,14,R,DE L'HOPITAL,NaN,...,1,Etablissement public de santé,1901-01-01,1901-01-01,2018-01-12,NaN,837272.3,6539470.4,"2,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
3,010000081,010780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,NaN,NaN,160,R,MARC PANISSOD,NaN,...,1,Etablissement public de santé,1901-01-01,1901-01-01,2020-02-04,NaN,935201.9,6584824.4,"1,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
4,010000099,010780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,NaN,NaN,13,AV,DU DOCTEUR BOYER,NaN,...,1,Etablissement public de santé,1945-01-01,1945-01-01,2020-06-30,NaN,870112.6,6536420.5,"2,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,LAMBERT_93",2022-11-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95741,980502389,590799730,SAMSAH TUTUBIYA - ALEFPA,NaN,NaN,NaN,390,R,DES FRANGIPANIERS,NaN,...,NaN,NaN,2022-06-08,2021-12-20,2022-07-19,NaN,521091.5,8572951.3,"3,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04
95742,980502405,980500854,PLATEFORME SENSORIELLE - ADSM,NaN,NaN,NaN,4,R,DE LA SIM,NaN,...,NaN,NaN,2022-02-26,2021-04-12,2022-07-20,NaN,514540.0,8585984.0,"2,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04
95743,980502439,980500557,SEPRODOM MAYOTTE,SEPRODOM MAYOTTE,NaN,NaN,NaN,CHE,MAVIKI,NaN,...,0,Non concerné,2020-12-01,2020-10-13,2022-10-06,NaN,520063.8,8580141.1,"3,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04
95744,980502454,980502447,LE CONFORT MEDICAL,NaN,NaN,NaN,100,RTE,DES BADAMIERS,C,...,9,indéterminé,2022-09-16,2022-09-16,2022-10-06,NaN,530514.6,8587199.1,"2,ATLASANTE,100,IGN,BD_ADRESSE,V2.2,UTM_S38",2022-11-04


In [10]:
clean_df.sample().T

,69461
nofinesset,750045676
nofinessej,750721391
rs,SAVS CHAMPIONNET
rslongue,SERVICE D ACCOMPAGNEMENT A LA VIE SOCIALE
complrs,NaN
compldistrib,74-76
numvoie,74
typvoie,R
voie,CHAMPIONNET
compvoie,NaN


In [11]:
clean_df["siret"]

0        26010004500012
1        26010003700068
2        26010028400017
3        26010010200011
4        26010013600019
              ...      
95741    77562407502209
95742               NaN
95743               NaN
95744               NaN
95745               NaN
Name: siret, Length: 95746, dtype: object

## Vérification de la qualité des données

In [12]:
intersection = pd.Series(np.intersect1d(structures.nofinesset.values, geolocalisations.nofinesset.values))

intersection.shape

(95746,)

In [13]:
only_structures = (structures
    [ ~structures.nofinesset.isin(intersection) ]
)

only_structures

,nofinesset,nofinessej,rs,rslongue,complrs,compldistrib,numvoie,typvoie,voie,compvoie,...,siret,codeape,codemft,libmft,codesph,libsph,dateouv,dateautor,maj,numuai


In [14]:
only_geolocalisations = (geolocalisations
    [ ~geolocalisations.nofinesset.isin(intersection) ]
)

only_geolocalisations

,nofinesset,coordxet,coordyet,sourcecoordet,datemaj


In [15]:
geolocalisations_missing = []

## Export final

In [16]:
clean_df.to_csv('finess-clean.csv', encoding='utf-8')